# Cleaning Data

In [151]:
import pandas as pd
import numpy as np

Load Dataset

In [152]:
player = 'cole'
data = pd.read_csv('./data/' + player + '.csv')
print(data)

       Unnamed: 0 pitch_type   game_date  release_speed  release_pos_x  \
0               0         KC  2018-10-14           80.6          -2.04   
1               1         SL  2018-10-14           88.0          -2.01   
2               2         CH  2018-10-14           86.1          -2.23   
3               3         CH  2018-10-14           87.5          -2.14   
4               4         KC  2018-10-14           80.5          -2.04   
...           ...        ...         ...            ...            ...   
21489        5407         SL  2019-03-29           89.8          -2.35   
21490        5408         FF  2019-03-29           96.3          -2.16   
21491        5409         SL  2019-03-29           88.5          -2.34   
21492        5410         FT  2019-03-29           96.7          -2.34   
21493        5411         FF  2019-03-29           96.2          -2.24   

       release_pos_z   player_name  batter  pitcher     events  ... fld_score  \
0               5.61  Cole, Ge

/home/logan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Simplify the batters on base columns:

In [153]:
data['on_base'] = np.sum([
    pd.notnull(data['on_1b']),
    pd.notnull(data['on_2b']),
    pd.notnull(data['on_3b'])
], axis=0)

In the data we have, the pitch (label) and the results of the pitch are in the same row. We would like the label to be on the same row as the previous pitch data, so we need to shift some of the columns down one row. 

In [154]:
to_shift_down = [
    'events',
    'zone',
    'type',
    'bb_type',
    'plate_x',
    'plate_z'
]

groups = data.groupby(['game_pk', 'inning'])

list_of_dfs = []
for _, g in groups:
    list_of_dfs.append(g)

for i, df in enumerate(list_of_dfs):
    for column in df:
        if column in to_shift_down:
            df["prev_" + column] = df[column].shift(periods=1)
    list_of_dfs[i] = df   
            
cleaned_data = pd.concat(list_of_dfs, axis=0, sort=False)

for col in to_shift_down:
    cleaned_data.pop(col)

Now, we explicitly list the columns that we want to keep:

In [155]:
keep_columns = [
    'pitch_type',
    'release_speed',
    'stand',
    'balls',
    'strikes',
    'plate_x',
    'plate_y',
    'plate_z',
    'outs_when_up',
    'inning',
    'game_pk',
    'pitch_num',
    'bat_score',
    'fld_score',
    'on_base',
    'prev_type',
    'prev_bb_type',
    'prev_plate_x',
    'prev_plate_z'
]

Delete the unnecessary columns:

In [156]:
for column in cleaned_data:
    if column not in keep_columns:
        cleaned_data.pop(column)

print("Kept Columns:")
for column in cleaned_data:
    print(column)

Kept Columns:
pitch_type
release_speed
stand
balls
strikes
outs_when_up
inning
game_pk
bat_score
fld_score
on_base
prev_type
prev_bb_type
prev_plate_x
prev_plate_z


Change some of the kept data to one-hot format:

In [157]:
change_to_oh = [
    'stand',
    'prev_type',
    'prev_bb_type',
]

In [158]:
for oh_column in change_to_oh:
    drop_first = (oh_column == 'stand')
    dummies = pd.get_dummies(cleaned_data[oh_column], drop_first=drop_first, prefix=oh_column)
    cleaned_data = pd.concat([cleaned_data, dummies], axis=1)
    cleaned_data.pop(oh_column)
    
print(cleaned_data)

      pitch_type  release_speed  balls  strikes  outs_when_up  inning  \
16064         FF           97.7      2        2             2       1   
16065         FF           99.6      1        2             2       1   
16066         SI           97.4      1        1             2       1   
16067         FF           97.9      0        1             2       1   
16068         FF           98.7      0        0             2       1   
...          ...            ...    ...      ...           ...     ...   
16093         SL           89.1      0        0             1       5   
16094         KC           82.9      1        1             0       5   
16095         FF           96.1      1        0             0       5   
16096         FF           97.1      0        0             0       5   
16082         KC           83.0      0        0             0       6   

       game_pk  bat_score  fld_score  on_base  prev_plate_x  prev_plate_z  \
16064   347712          0          0        2 

Save the data:

In [159]:
cleaned_data.to_csv('./cleaned_data/' + player + '.csv')

Write data to `.npy` files

In [160]:
labels = cleaned_data.pop('pitch_type')

In [161]:
l = labels.astype('category').cat.codes
np.save('./cleaned_data/cole_labels.npy', l)
l

16064    3
16065    3
16066    8
16067    3
16068    3
        ..
16093    9
16094    6
16095    3
16096    3
16082    6
Length: 21494, dtype: int8

In [162]:
np.save('./cleaned_data/cole_data.npy', cleaned_data)
cleaned_data

,release_speed,balls,strikes,outs_when_up,inning,game_pk,bat_score,fld_score,on_base,prev_plate_x,prev_plate_z,stand_R,prev_type_B,prev_type_S,prev_type_X,prev_bb_type_fly_ball,prev_bb_type_ground_ball,prev_bb_type_line_drive,prev_bb_type_popup
16064,97.7,2,2,2,1,347712,0,0,2,NaN,NaN,0,0,0,0,0,0,0,0
16065,99.6,1,2,2,1,347712,0,0,2,0.71,2.55,0,0,0,1,0,0,0,1
16066,97.4,1,1,2,1,347712,0,0,2,0.49,3.54,0,1,0,0,0,0,0,0
16067,97.9,0,1,2,1,347712,0,0,2,-0.12,2.83,0,0,1,0,0,0,0,0
16068,98.7,0,0,2,1,347712,0,0,2,1.24,2.01,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16093,89.1,0,0,1,5,635920,0,1,0,-0.29,3.01,1,0,1,0,0,0,0,0
16094,82.9,1,1,0,5,635920,0,1,0,0.69,2.97,0,0,1,0,0,0,0,0
16095,96.1,1,0,0,5,635920,0,1,0,-0.47,2.55,0,0,0,1,1,0,0,0
16096,97.1,0,0,0,5,635920,0,1,0,-0.89,3.02,0,0,1,0,0,0,0,0
